In [2]:
import pandas as pd
import numpy as np

pd.set_option('max_rows', 10)

DATA_FILE_PATH_WIN = 'D:/analytics/stock/hist'
TEST_DATA_FILE_PATH = 'D:/analytics/stock/hist/002415.csv'

df = pd.read_csv(TEST_DATA_FILE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-04-22,31.17,2.37,30.686,30.857,30.809
2016-04-21,30.45,0.79,30.732,30.806,30.803
2016-04-20,30.21,-1.60,30.880,30.831,30.875
2016-04-19,30.70,-0.65,31.064,30.946,30.953
2016-04-18,30.90,-1.59,31.004,31.016,31.038
...,...,...,...,...,...
2013-05-03,37.00,0.00,37.260,37.260,37.260
2013-05-02,37.00,2.15,37.325,37.325,37.325
2013-04-26,36.22,-1.79,37.433,37.433,37.433


In [14]:
r = df.resample('W')
r['p_change'].mean().dropna().sort_values()

date
2015-07-12   -5.413333
2015-06-21   -3.862000
2013-12-08   -2.440000
2015-03-08   -2.382500
2014-03-23   -2.336000
                ...   
2015-04-05    2.442000
2013-06-16    2.645000
2014-12-07    2.842000
2015-01-11    3.418000
2015-05-24    6.086000
Name: p_change, dtype: float64